This notebook is going to take you through an analysis of platformer games in the VGLC [1] as well as a custom dataset made for the housed in the repository where this notebook resides. Before going through the details, we are going to read everything in. Each platformer will be read into a [n-gram](https://en.wikipedia.org/wiki/N-gram) based off the previous work from Dahlskog et al. which broke levels into columns as input [2].

In [1]:
%matplotlib inline
from scipy.stats import mode
from pathlib import Path
from enum import Enum

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os

We have two cases. The first is the custom format which is a json array. The second is the VGLC format which is a text file, very similar to a JSON array. For both we read htem in and return the contents of the files.

In [2]:
def read_json(file_path):
    f = open(file_path)
    content = f.read()
    f.close()
    
    level_matrix = json.loads(content)
    columns = [[] for i in range(len(level_matrix[0]))]

    for row in level_matrix:
        for i in range(len(row)):
            columns[i].append(row[i])

    return columns

In [3]:
def read_txt(file_path, vertical):
    f = open(file_path)
    lines = f.readlines()
    f.close()
    
    if vertical:
        columns = []
        for line in reversed(lines):
            columns.append(list(line.strip()))
    else:
        columns = [[] for _ in range(len(lines[0]) - 1)]
        for line in lines:
            line = line.strip()
            for columnIndex in range(len(line)):
                columns[columnIndex].insert(0, line[columnIndex])
            
    return columns

In [4]:
def get_file_contents(path, vertical=False, json=False):
    file_contents = []
    for file_name in os.listdir(path):
        if 'meta' in file_name:
            continue
        
        file_path = os.path.join(path, file_name)
        if os.path.isdir(file_path):
            continue
        
        if json:
            file_contents.append(read_json(file_path))
        else:
            file_contents.append(read_txt(file_path, vertical))
    
    return file_contents

I have the [VGLC](https://github.com/TheVGLC/TheVGLC) stored in `~/data/TheVGLC`. If you have it somewhere else then you will need to modify the code block below to use the correct path. Also note that there is a vertical flag since [Kid Icarus levels are in a vertical format](https://github.com/TheVGLC/TheVGLC/blob/master/Kid%20Icarus/Processed/kidicarus_1.txt).

In [5]:
def get_vglc_path(game_name):
    return os.path.join(Path.home(), 'data', 'TheVGLC', game_name, 'Processed')

In [6]:
class Game(Enum):
    SuperMarioBros = 0
    SuperMarioBros2 = 1
    SuperMarioBros2Japan = 2
    SuperMarioLand = 3
    KidIcarus = 4
    Custom = 5

In [7]:
data = {}
data[Game.SuperMarioBros] = get_file_contents(get_vglc_path("Super Mario Bros"))
data[Game.SuperMarioBros2Japan] = get_file_contents(get_vglc_path("Super Mario Bros 2 (Japan)"))
data[Game.SuperMarioLand] = get_file_contents(get_vglc_path("Super Mario Land"))
data[Game.KidIcarus] = get_file_contents(get_vglc_path("Kid Icarus"), vertical=True)
data[Game.Custom] = get_file_contents(os.path.join('..', 'Assets', 'Resources', 'Levels'), json=True)

path = os.path.join(get_vglc_path("Super Mario Bros 2"), 'WithEnemies')
data[Game.SuperMarioBros2] = get_file_contents(path)

Now that we have loaded in all the games we are going to build them into n-grams. We'll do n-grams of 2 to 5.

In [8]:
min_n = 2
max_n = 5

In [9]:
def build_n_gram(n, levels):
    ngram = {}
    
    for columns in levels:
        prior = []
        for col in columns:
            json_col = json.dumps(col)
            if len(prior) == n - 1:
                json_prior = json.dumps(prior)
                
                if json_prior not in ngram:
                    ngram[json_prior] = {}
                    ngram[json_prior][json_col] = 1
                elif json_col not in ngram[json_prior]:
                    ngram[json_prior][json_col] = 1
                else:
                    ngram[json_prior][json_col] += 1
                
                prior.pop(0)

            prior.append(col)

    return ngram

n_grams = {}
for n in range(min_n, max_n + 1):
    n_gram_data = {}
    
    for game in Game:
        n_gram_data[game] = build_n_gram(n, data[game])
        
    n_grams[n] = n_gram_data

First, we look at the number of priors versus the number of outputs.

In [10]:
def build_prior_and_num_outputs_df(n):
    game_to_n_gram = n_grams[n]
    rows = []
    for game in game_to_n_gram:
        n_gram = game_to_n_gram[game]
        
        priors = len(n_gram)
        output_count = 0
        for prior in n_gram:
            for output in n_gram[prior]:
                output_count += n_gram[prior][output]
                
        rows.append({
            'Game': str(game), 
            'Prior Count': priors, 
            'Number of Outputs': output_count,
            'Number of Outputs / Prior': output_count / priors})
    
    return pd.DataFrame(rows)

prior_and_num_output_dfs = {}
for n in n_grams:
    print(f'n = {n}')
    prior_and_num_output_dfs[n] = build_prior_and_num_outputs_df(n)
    print(prior_and_num_output_dfs[n].to_string(index=False), '\n\n\n')

n = 2
                      Game  Prior Count  Number of Outputs  Number of Outputs / Prior
       Game.SuperMarioBros          271               2908                  10.730627
      Game.SuperMarioBros2          217               3289                  15.156682
 Game.SuperMarioBros2Japan          604               4497                   7.445364
       Game.SuperMarioLand          631               2787                   4.416799
            Game.KidIcarus          348               1249                   3.589080
               Game.Custom          217                824                   3.797235 



n = 3
                      Game  Prior Count  Number of Outputs  Number of Outputs / Prior
       Game.SuperMarioBros          681               2893                   4.248164
      Game.SuperMarioBros2          575               3273                   5.692174
 Game.SuperMarioBros2Japan         1320               4475                   3.390152
       Game.SuperMarioLand         122

At n = 2 we have a min of 217 priors and a max of 604 priors. At n = 5 the min is 582 whereas the max is 2452. We expect to see the number of priors rise with an increase to n since the larger the prior the more unique it should be.

We see that the numer of outputs reduces as n increases due to the dataset's size being less effective as n increases. We would need more data to make up for this. Lastly the ratio shows the number of outputs of the number of priors. We want this number to be high since it means that priors will have more associated data with it.

Unfortunately, this ratio is misleading. It shows the average across all priors. It could be that one prior is associated with the majority of outputs and the rest only have one. 

In [11]:
def n_gram_prior_stats(n):
    game_to_n_gram = n_grams[n]
    rows = []
    
    for game in game_to_n_gram:
        n_gram = game_to_n_gram[game]
        outputs = []
        
        for prior in game_to_n_gram[game]:
            outputs.append(len(n_gram[prior]))
            
        outputs = np.array(outputs)

        data = {}
        data['Game'] = str(game)
        data['Mean'] = np.mean(outputs)
        data['Median'] = np.median(outputs)
        data['Mode'] = mode(outputs)[0][0]
        data['Min'] = np.min(outputs)
        data['Max'] = np.max(outputs)
        data['STD'] = np.std(outputs)
        data['Prios with 1 Output / Num Priors'] = np.count_nonzero(outputs == 1) / len(outputs)
        rows.append(data)
        
    return pd.DataFrame(rows) 

for n in n_grams:
    print(f'n = {n}')
    print(n_gram_prior_stats(n).to_string(index=False), '\n\n\n')

n = 2
                      Game      Mean  Median  Mode  Min  Max       STD  Prios with 1 Output / Num Priors
       Game.SuperMarioBros  2.512915     1.0     1    1   58  4.838234                          0.535055
      Game.SuperMarioBros2  2.649770     1.0     1    1   47  4.752580                          0.585253
 Game.SuperMarioBros2Japan  2.192053     1.0     1    1   60  4.353145                          0.607616
       Game.SuperMarioLand  1.941363     1.0     1    1   55  3.124676                          0.670365
            Game.KidIcarus  1.882184     1.0     1    1   99  5.429318                          0.729885
               Game.Custom  1.834101     1.0     1    1   24  2.304101                          0.654378 



n = 3
                      Game      Mean  Median  Mode  Min  Max       STD  Prios with 1 Output / Num Priors
       Game.SuperMarioBros  1.547724     1.0     1    1   50  2.398206                          0.794420
      Game.SuperMarioBros2  1.638261   

What we're now looking at is a case by base basis for each prior. For example, at n = 2 for Super Mario Bros., we see that the average prior will have 2.51 potential outputs. This is good. It means that our dataset is not too heavily favoring priors to one output. We want to discourage this one to one formula since that means the n-gram is memorizing. More generally, a machine learning algorithm is not receiving enough input for the data. 

The average is deceptive for Super Mario Bros., though. Look at the median, mode, and max columns. They former two are 1 and the latter is 58. The mode tells us that the majority of our priors are only assocaited with a single output. The generation with an n-gram is guaranteed to follow one path. The generation will not be unique, it will be memorized. If the point of PCG is to generate new content, then our dataset is preventing us.

Also note that we only looked at one example at n=2. Looking further you can see that the mode for every game is 1. Furthermore, the problem only gets worse as n is incremented. The larger the n, the smaller the input size and the more unique priors will be. As a result, we see the max number of outputs steadily decrease given a larger dataset. The question becomes, how can we solve this?

We can come up with a simplification method. We take the levels and break them down into smaller bits. Already, we have taken levels from 2D platformers and turne them from matrices to an array of IDs which represent columns. We can simplify even further without losing information. Looking at the work from Shaker et al. [3], we can break columns into a set of patterns. In their work, they built a grammar that was then evolved. In our case we can can simplify the entire dataset of columns into categories:

1. Linear
2. Linear with enemy
3. Platform forced
4. Platform optional
5. Platform optional enemy
6. Platform forced enemy

You'll notice that this set of categories does not work with *Kid Icarus*. As a result, it won't be used going forward but does show that games thare not 2d, linear platformers will need a different simplification.

In [12]:
Linear = 'l'
LinearEnemy = 'le'
PlatformForced = 'pf'
PlatformOptional = 'po'
PlatformOptionalEnemy = 'poe'
PlatformForcedEnemy = 'pfe'

In [13]:
def simplify_vglc(levels):
    '''
    VGLC levels used all have the assumption that the bottom will be a brick
    or not. This is not a perfect simplification since you may argue that 
    somethinge like what is below would be not result in platform forced:

    xxxxxxxxxxxx
    ------------

    This is a platform forced since it is above the base level. 
    '''
    simplified_levels = []
    for level in levels:
        simplified_level = []
        for col in level:
            column_has_enemy = 'e' in [c.lower() for c in col]

            if col[0] == '-' or (col[0].lower() == 'x' and col[1] != '-' and (col[1].lower() != 'e')):
                if column_has_enemy:
                    simplified_level.append(PlatformForcedEnemy)
                else:
                    simplified_level.append(PlatformForced)
            else:
                if all(tile == '-' or tile.lower() == 'e' for tile in col[1:]):
                    if column_has_enemy:
                        simplified_level.append(LinearEnemy)
                    else:
                        simplified_level.append(Linear)
                else:
                    if column_has_enemy:
                        simplified_level.append(PlatformOptionalEnemy)
                    else:
                        simplified_level.append(PlatformOptional)

        simplified_levels.append(simplified_level)

    return simplified_levels

In [14]:
def simplify_custom(levels):
    '''
    custom levels have different tiles and the base ground level is at index 39.
    '''
    simplified_levels = []
    enemies = ['A', 'B', 'C', 'D']

    for level in levels:
        simplified_level = []
        for col in level:
            column_has_enemy = False
            col_1_is_enemy = False

            for enemy in enemies:
                column_has_enemy &= enemy in col
                col_1_is_enemy |= col[1] == enemy

            if col[39] == ' ' or (col[39] == 'b' and col[1] != ' ' and not col_1_is_enemy):
                if column_has_enemy:
                    simplified_level.append(PlatformForcedEnemy)
                else:
                    simplified_level.append(PlatformForced)
            else:
                spliced_column = col[:39] + col[:38] # removes the base column
                if all((tile == ' ' or tile == 's' or tile == '$' or tile in enemies) for tile in spliced_column):
                    if column_has_enemy:
                        simplified_level.append(LinearEnemy)
                    else:
                        simplified_level.append(Linear)
                else:
                    if column_has_enemy:
                        simplified_level.append(PlatformOptionalEnemy)
                    else:
                        simplified_level.append(PlatformOptional)

        simplified_levels.append(simplified_level)

    return simplified_levels

In [15]:
simplified_data = {}
simplified_data[Game.SuperMarioBros] = simplify_vglc(data[Game.SuperMarioBros])
simplified_data[Game.SuperMarioBros2Japan] = simplify_vglc(data[Game.SuperMarioBros2Japan])
simplified_data[Game.SuperMarioLand] = simplify_vglc(data[Game.SuperMarioLand])
simplified_data[Game.Custom] = simplify_custom(data[Game.Custom])

In [16]:
n_grams = {}
for n in range(min_n, max_n + 1):
    n_gram_data = {}
    
    for game in simplified_data:
        n_gram_data[game] = build_n_gram(n, simplified_data[game])
        
    n_grams[n] = n_gram_data

In [17]:
prior_and_num_output_dfs = {}
for n in n_grams:
    print(f'n = {n}')
    prior_and_num_output_dfs[n] = build_prior_and_num_outputs_df(n)
    print(prior_and_num_output_dfs[n].to_string(index=False), '\n\n\n')

n = 2
                      Game  Prior Count  Number of Outputs  Number of Outputs / Prior
       Game.SuperMarioBros            6               2908                 484.666667
 Game.SuperMarioBros2Japan            6               4497                 749.500000
       Game.SuperMarioLand            6               2787                 464.500000
               Game.Custom            3                824                 274.666667 



n = 3
                      Game  Prior Count  Number of Outputs  Number of Outputs / Prior
       Game.SuperMarioBros           31               2893                  93.322581
 Game.SuperMarioBros2Japan           34               4475                 131.617647
       Game.SuperMarioLand           29               2778                  95.793103
               Game.Custom            9                809                  89.888889 



n = 4
                      Game  Prior Count  Number of Outputs  Number of Outputs / Prior
       Game.SuperMarioBros  

text here

In [18]:
for n in n_grams:
    print(f'n = {n}')
    print(n_gram_prior_stats(n).to_string(index=False), '\n\n\n')

n = 2
                      Game      Mean  Median  Mode  Min  Max       STD  Prios with 1 Output / Num Priors
       Game.SuperMarioBros  5.166667     5.5     6    3    6  1.067187                               0.0
 Game.SuperMarioBros2Japan  5.666667     6.0     6    5    6  0.471405                               0.0
       Game.SuperMarioLand  4.833333     5.0     5    3    6  1.067187                               0.0
               Game.Custom  3.000000     3.0     3    3    3  0.000000                               0.0 



n = 3
                      Game      Mean  Median  Mode  Min  Max       STD  Prios with 1 Output / Num Priors
       Game.SuperMarioBros  3.354839     3.0     3    1    6  1.492960                          0.161290
 Game.SuperMarioBros2Japan  3.205882     3.0     2    1    6  1.548802                          0.147059
       Game.SuperMarioLand  2.689655     2.0     2    1    6  1.440868                          0.241379
               Game.Custom  2.666667   

text here

### Sources

1. Summerville, A. J., Snodgrass, S., Mateas, M., & Ontanón, S. (2016). The vglc: The video game level corpus. arXiv preprint arXiv:1606.07487.
2. Dahlskog, S., Togelius, J., & Nelson, M. J. (2014, November). Linear levels through n-grams. In Proceedings of the 18th International Academic MindTrek Conference: Media Business, Management, Content & Services (pp. 200-206).
3. Shaker, N., Nicolau, M., Yannakakis, G. N., Togelius, J., & O'neill, M. (2012, September). Evolving levels for super mario bros using grammatical evolution. In 2012 IEEE Conference on Computational Intelligence and Games (CIG) (pp. 304-311). IEEE.